## Libraries

In [40]:
import sqlite3 # connect to .db file
import sql_exec # python module to run queries and format output tables for sql mystery db
import pandas as pd # dataframe module

## Database Interaction

In [45]:
# creating file path
dbfile = 'sql-murder-mystery.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

meta = get_metadata(cur)

# Step 1: Query the crime scene reports to assert that there is a record of a murder taking place on January 15, 2018 in SQL City.
query1 = "SELECT * FROM crime_scene_report WHERE date==20180115 AND city=='SQL City';"
r = sql_exec.run_query(cur, query1)
df1 = sql_exec.format_output(meta, r, query1)

# Step 2: Join the interview table with the persons table to view the transcripts of the witness interviews paired with the witness names.
query2 = "SELECT p.name, p.id, i.transcript FROM person p JOIN interview i ON i.person_id == p.id WHERE p.id IN (14887, 16371);"
r = sql_exec.run_query(cur, query2)
df2 = sql_exec.format_output(meta, r, query2)

# Step 3: Find the names and IDs of people who were at the gym on January 9th, 2018, and who are gold members with member IDs that start with “48Z”.
query3 = "SELECT gfnm.id, gfnm.person_id, gfnm.name, gfnm.membership_status, gfnchi.check_in_date FROM get_fit_now_member gfnm JOIN get_fit_now_check_in gfnchi ON gfnm.id==gfnchi.membership_id WHERE membership_status=='gold' AND gfnchi.check_in_date==20180109  AND gfnm.id LIKE '48Z%';"
r = sql_exec.run_query(cur, query3)
df3 = sql_exec.format_output(meta, r, query3)

# Step 4: Check if either of the suspects are registered to a vehicle thatwas the getaway car. The getaway car had ‘H42W’ in its license plate number.
query4 = "SELECT p.name, p.id, dl.plate_number, dl.car_make, dl.car_model FROM drivers_license dl JOIN person p ON dl.id == p.license_id WHERE dl.plate_number LIKE '%H42W%'"
r = sql_exec.run_query(cur, query4)
df4 = sql_exec.format_output(meta, r, query4)

# Step 5: For further investigation, search for interview transcripts from Jeremy Bowers.
query5 = "SELECT p.name, p.id, i.transcript FROM interview i JOIN person p ON p.id==i.person_id WHERE person_id==67318;"
r = sql_exec.run_query(cur, query5)
df5 = sql_exec.format_output(meta, r, query5)

# Step 6: Search for the name of the woman who hired Jeremy Bowers to commit the crime
query6 = """
            SELECT p.id, p.name, dl.height, dl.hair_color, dl.car_make,
            dl.car_model, fec.event_name, fec.date
            FROM person p
            JOIN drivers_license dl ON dl.id==p.license_id
            JOIN facebook_event_checkin fec ON fec.person_id==p.id
            WHERE dl.height >= 65 AND dl.height <= 67
            AND dl.gender=='female' AND dl.hair_color=='red'
            AND dl.car_make=='Tesla' AND dl.car_model=='Model S'
            AND fec.event_name LIKE '%SQL Symphony Concert%';
        """
r = sql_exec.run_query(cur, query6)
df6 = sql_exec.format_output(meta, r, query6)


# Be sure to close the cursor
cur.close()
# Be sure to close the connection
con.close()

## Step #1
```
We are able to discover that the incident in question had two witnesses. The first witness lives in the last house on Northwestern Dr, and the second witness is named Annabel, and she lives on Franklin Ave. Our next task should be to read the interviews of the witnesses to collect more information on the murder suspect(s).
```


In [43]:
df1

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


**Figure 1:** We know that the crime was a murder so it is clear that the highlighted record
describes the incident.

## Step #2

```
From the resulting output we are able to read the transcripts of the witness reports. We learn several important insights from these interviews. I have listed new information  regarding the murder suspect below.
```
**Suspect Information:**

1. The suspect is a man

2. The suspect had a “Get Fit Now Gym” bag

3. The suspect’s bag had an ID number starting with “48Z”

4. The suspect was likely a gold member to the “Get Fit Now Gym”

5. The suspect got into a car that had the phrase “H42W” as a subset of the license plate number

6. The suspect was at the “Get Fit Now Gym” on January 9th, 2018

In [44]:
df2

,person.name,person.id,interview.transcript
0,Morty Schapiro,14887,I heard a gunshot and then saw a man run out. ...
1,Annabel Miller,16371,"I saw the murder happen, and I recognized the ..."


**Figure 2:** This table combines the name, person id, and corresponding transcript of all the witnesses to the murder.

## Step #3:
```
Find the names and IDs of people who were at the gym on
January 9th, 2018, and who are gold members with member IDs that
start with “48Z”.
```

In [46]:
df3

,get_fit_now_member.id,get_fit_now_member.person_id,get_fit_now_member.name,get_fit_now_member.membership_status,get_fit_now_check_in.check_in_date
0,48Z7A,28819,Joe Germuska,gold,20180109
1,48Z55,67318,Jeremy Bowers,gold,20180109


**Figure 3:** This output table holds records of people that match all of the criteria outlined in the witness interviews. It is safe to say that either Joe Germuska or Jeremy Bowers committed the murder.

## Step #4:

```
In this step, we can definitively say that Jeremy Bowers matched all of the criteria outlined by the witnesses, and he has registered a vehicle that has a license plate containing ‘H42W’ which was part of a witness testimony. This is enough evidence to reasonably accuse Jeremy Bowers of the murder
```

In [47]:
df4

,person.name,person.id,drivers_license.plate_number,drivers_license.car_make,drivers_license.car_model
0,Tushar Chandra,51739,4H42WR,Nissan,Altima
1,Jeremy Bowers,67318,0H42W2,Chevrolet,Spark LS
2,Maxine Whitely,78193,H42W0X,Toyota,Prius


**Figure 2:** This output table contains the names and id numbers of all the people who are
registered to vehicles with the ‘H42W’ substring in their license plate numbers.

## Step #5

```
From the Jeremy Bowers transcript we learn several key insights regarding the circumstances of his crime. We learn that he was hired by a woman who had a lot of money, who was between 65 and 67 inches tall, with red hair, drives a Tesla Model S, and attended the SQL Symphony Concert three times in December 2017.
```

In [48]:
df5

,person.name,person.id,interview.transcript
0,Jeremy Bowers,67318,I was hired by a woman with a lot of money. I ...


**Figure 5:** This output table contains the transcript from the interview from Jeremy Bowers.

## Step #6

```
Based on the description in the Jeremy Bowers interview we are able to find who was the brains behind the operation
```

In [49]:
df6

,person.id,person.name,drivers_license.height,drivers_license.hair_color,drivers_license.car_make,drivers_license.car_model,facebook_event_checkin.event_name,facebook_event_checkin.date
0,99716,Miranda Priestly,66,red,Tesla,Model S,SQL Symphony Concert,20171206
1,99716,Miranda Priestly,66,red,Tesla,Model S,SQL Symphony Concert,20171212
2,99716,Miranda Priestly,66,red,Tesla,Model S,SQL Symphony Concert,20171229


**Figure 6:** This table shows three instances of Jeremy Bowers’ boss, Miranda Priestly, attending the SQL Symphony Concert in December of 2017. 

## Conclusion

```
We have gathered enough evidence to accuse Jeremy Bowers of the murder in SQL City on January 15, 2018. Upon further investigation it has become clear that he was not acting alone, and that he was put up to the crime by a red haired woman named Miranda Priestly. The motive behind the crime is unclear, other than that Miranda is a wealthy woman and she likely wanted to set up Jeremy with the blame in case the authorities were able to solve the crime.
```